<a href="https://colab.research.google.com/github/Wessi/N-gram-Amharic/blob/main/NDVI_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
merged_data = pd.read_excel('/content/drive/MyDrive/merged_ndvi_2017_2023.xlsx')


In [6]:
merged_data.head(5).to_csv('top_5_rows.csv', index=False)

In [ ]:
# Step 1: Generate new column names as NDVI_1, NDVI_2, ..., NDVI_133
new_columns = [f'NDVI_{i+1}' for i in range(133)]

# Step 2: Rename the NDVI columns in the merged_data DataFrame
merged_data.columns = ['GRID_CODE'] + new_columns

# Step 3: Verify that the column names have been updated
print(merged_data.columns)


In [ ]:
merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
df_dropped = merged_data.drop(columns=merged_data.filter(like='_2023').columns)
df_dropped.head()

In [ ]:
merged_data.head(1)

In [ ]:
merged_data.drop('bbox', axis=1, inplace=True)

In [ ]:
merged_data.drop('2021CPSZs', axis=1, inplace=True)

merged_data.head()

In [ ]:
merged_data.info()

In [ ]:
merged_data.shape

In [ ]:
null_values = merged_data.isnull().sum()
null_columns = null_values[null_values > 0]
print("\nColumns with missing values:\n", null_columns)

**Average NDVI Values for each Grids**

In [ ]:
ndvi_columns = ['NDVI_' + str(i) for i in range(17, 36)]
merged_data['Average_NDVI'] = merged_data[ndvi_columns].mean(axis=1)
average_ndvi_per_grid = merged_data.groupby('GRID_CODE')['Average_NDVI'].mean().reset_index()

average_ndvi_per_grid.head()

**NDVI Variability for ach Grid**

In [ ]:

merged_data['Variability'] = merged_data[ndvi_columns].std(axis=1)
std_for_each_grid = merged_data[['GRID_CODE', 'Variability']]
std_for_each_grid.head()

**Peak NDVI for each Grid**

In [ ]:
merged_data['Peak'] = merged_data[ndvi_columns].max(axis=1)
peak_for_each_grid = merged_data[['GRID_CODE', 'Peak']]
peak_for_each_grid.head()

**1. Precision Factor**

In [ ]:
merged_data['precision_factor'] = peak_for_each_grid['Peak'] / average_ndvi_per_grid['Average_NDVI']

precision_factor_for_each_grid = merged_data[['GRID_CODE', 'precision_factor']]
precision_factor_for_each_grid.head()

In [ ]:
precision_factor_for_each_grid.describe()

**2. Inclusion Factor**

In [ ]:
regional_benchmark = average_ndvi_per_grid['Average_NDVI'].mean()
print(regional_benchmark)

In [ ]:
merged_data['inclusion_factor'] = regional_benchmark/ std_for_each_grid['Variability']

inclusion_factor_for_each_grid = merged_data[['GRID_CODE', 'inclusion_factor']]
inclusion_factor_for_each_grid.head()

In [ ]:
inclusion_factor_for_each_grid.describe()

**3. Consistency Factor**

In [ ]:
merged_data['coefficient_of_variablity'] = std_for_each_grid['Variability']/ average_ndvi_per_grid['Average_NDVI']

coefficient_of_variablity_for_each_grid = merged_data[['GRID_CODE', 'coefficient_of_variablity']]
coefficient_of_variablity_for_each_grid.head()

In [ ]:
merged_data['consistency_factor'] = 1/merged_data['coefficient_of_variablity']
consistency_factor_for_each_grid = merged_data[['GRID_CODE', 'consistency_factor']]
consistency_factor_for_each_grid.head()

In [ ]:
consistency_factor_for_each_grid.describe()

**4. Optimization Factor**

In [ ]:
merged_data = merged_data.drop(columns=['GRID-CODE'], errors='ignore')
scaler = MinMaxScaler(feature_range=(0, 1))
ndvi_train_scaled = scaler.fit_transform(merged_data[ndvi_columns])

In [ ]:

def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)

n_steps = 6

X_train, y_train = create_sequences(ndvi_train_scaled, n_steps)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(n_steps, len(ndvi_columns)), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dense(len(ndvi_columns)))
model.compile(optimizer='adam', loss='mse')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
print(X_train)

In [ ]:
import pandas as pd
import numpy as np

# Assuming `merged_data` contains columns: 'GRID_CODE', 'NDVI_1', 'NDVI_2', ..., 'NDVI_152'
# Load your merged_data DataFrame
# merged_data = pd.read_csv('path_to_your_data.csv')

# Extract NDVI columns
ndvi_columns = [f'NDVI_{i}' for i in range(1, 32)]  # Assuming 152 columns (8 years * 19 values per year)
grid_codes = merged_data['GRID_CODE'].values
ndvi_data = merged_data[ndvi_columns].values  # Shape: (num_grids, 152)

def create_sequences(data, input_steps=133, output_steps=19):
    """
    Prepare sequences where inputs are the past 133 values (7 years)
    and outputs are the next 19 values (1 year).
    """
    X, y = [], []
    for row in data:
        # Use the first 133 NDVI values as input
        X.append(row[:input_steps])
        # Use the next 19 NDVI values as output
        y.append(row[input_steps:input_steps + output_steps])
    return np.array(X), np.array(y)

# Create input (X) and output (y) sequences
input_steps = 133  # 19 values * 7 years
output_steps = 19  # Predict 19 values for the next year
X, y = create_sequences(ndvi_data, input_steps, output_steps)

print(f"Input shape: {X.shape}, Output shape: {y.shape}")


In [ ]:
import pickle
from tensorflow.keras.models import model_from_json

# Save the model architecture and weights separately using pickle
def save_model_pickle(model, filename='ndvi_model.pkl'):
    # Step 1: Save the model architecture as JSON
    model_json = model.to_json()

    # Step 2: Save the model weights
    model_weights = model.get_weights()

    # Step 3: Use pickle to save both JSON and weights
    with open(filename, 'wb') as f:
        pickle.dump((model_json, model_weights), f)
    print(f"Model saved as {filename}")

# Assuming `model` is your trained model
save_model_pickle(model)


In [ ]:
model = load_model_pickle('ndvi_model.pkl')
y_pred = model.predict(X_test)


In [ ]:
y_pred.shape

In [ ]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import model_from_json
import tensorflow as tf

# Load the saved model using pickle
def load_model_pickle(filename='ndvi_model.pkl'):
    with open(filename, 'rb') as f:
        model_json, model_weights = pickle.load(f)
    loaded_model = model_from_json(model_json)
    loaded_model.set_weights(model_weights)
    return loaded_model

# Load the model
loaded_model = load_model_pickle('ndvi_model.pkl')

# Compile the loaded model
loaded_model.compile(optimizer='adam', loss='mean_squared_error')
print("Model loaded and compiled successfully.")


In [ ]:
def prepare_input_no_scaler(grid_code, past_ndvi_data, n_steps):
    """
    Prepare the input data using the provided GRID_CODE and past NDVI values without scaling.
    """
    if len(past_ndvi_data) != n_steps:
        raise ValueError(f"Expected {n_steps} NDVI values, but got {len(past_ndvi_data)}")

    # Create an array with repeated GRID_CODE and NDVI data for each timestep
    input_data = np.array([[grid_code] + past_ndvi_data])

    # Repeat the grid_code for each timestep and reshape to match the model's input shape
    X_input = np.tile(input_data, (n_steps, 1)).reshape(1, n_steps, -1)
    return X_input


In [ ]:
import pickle
from tensorflow.keras.models import Sequential, load_model

# Save the trained model
model.save('ndvi_model.h5')

# Save the scaler using pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Model and scaler saved successfully.")

In [ ]:
import matplotlib.pyplot as plt
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(training_loss, label='Training Loss', color='blue')
plt.plot(validation_loss, label='Validation Loss', color='orange')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

Testing

In [ ]:
ndvi_2024 = pd.read_excel('/content/drive/MyDrive/NDVI_2024.xlsx')
ndvi_2024.head()

In [ ]:
ndvi_2023 = pd.read_excel('/content/drive/MyDrive/NDVI_2023.xlsx')
ndvi_2023.head()

In [ ]:
ndvi_2024 = ndvi_2024.drop(columns=['bbox',	'GRID_CODE',	'2021CPSZs',	'NDVI_0',	'NDVI_1',	'NDVI_2',	'NDVI_3'])

In [ ]:
ndvi_2023 = ndvi_2023.drop(columns=['bbox',	'GRID_CODE',	'2021CPSZs',	'NDVI_0',	'NDVI_1',	'NDVI_2',	'NDVI_3', 'NDVI_4','NDVI_5','NDVI_6','NDVI_7','NDVI_8','NDVI_9','NDVI_10','NDVI_11','NDVI_12','NDVI_13','NDVI_14','NDVI_15','NDVI_16'])

In [ ]:
ndvi_2023.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
ndvi_2023_scaled = scaler.fit_transform(ndvi_2023)

In [ ]:
n_steps = 6
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)
X_test, y_test = create_sequences(ndvi_2023_scaled, n_steps)

In [ ]:
y_test_pred_scaled = model.predict(X_test)

y_test_pred = scaler.inverse_transform(y_test_pred_scaled)
y_test_actual = scaler.inverse_transform(y_test)
y_test_pred

In [ ]:
print(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))
mae = mean_absolute_error(y_test_actual, y_test_pred)
performance_metrics = {
    'RMSE': rmse,
    'MAE': mae
}

print("Performance Metrics on 2023:")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

In [ ]:
r2 = r2_score(y_test_actual, y_test_pred)
r2

In [ ]:
optimization_factor = r2/(rmse+mae)
print(optimization_factor)

In [ ]:

# Create sequences for training
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data) - n_steps_in - n_steps_out + 1):
        X.append(data[i:i + n_steps_in])
        y.append(data[i + n_steps_in:i + n_steps_in + n_steps_out])
    return np.array(X), np.array(y)

In [ ]:
# Parameters
n_steps_in = 15  # Look-back period
n_steps_out = 4  # Prediction window

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
ndvi_train_scaled = scaler.fit_transform(merged_data[ndvi_columns])

# Prepare training data
X_train, y_train = create_sequences(ndvi_train_scaled, n_steps_in, n_steps_out)
X_train = X_train.reshape((X_train.shape[0], n_steps_in, len(ndvi_columns)))

y_train = y_train[:,:,0] # Select the first feature from all time steps
y_train = y_train.reshape((y_train.shape[0], n_steps_out))


# Define and compile the model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(n_steps_in, len(ndvi_columns)), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dense(n_steps_out))  # Output layer for 4 future NDVI values
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# **PICO SCORE**

In [ ]:
def generate_base_score():
    return np.random.randint(300, 801)
base_score = generate_base_score()
print(f"Randomly generated base score: {base_score}")

In [ ]:
merged_data['pico_score'] =(merged_data['precision_factor'] + merged_data['inclusion_factor'] +  merged_data['consistency_factor'] +
                            optimization_factor)
pico_score_for_each_grid = merged_data[['GRID_CODE', 'pico_score',
                                        'precision_factor', 'inclusion_factor',
                                        'consistency_factor']]
pico_score_for_each_grid.head()

In [ ]:
# Find the minimum and maximum of the pico_score
min_score = merged_data['pico_score'].min()
max_score = merged_data['pico_score'].max()

# Define the new range
new_min = 300
new_max = 800

# Apply the mapping formula to map pico_score to the range of 300 to 800
merged_data['fico_score'] = ((merged_data['pico_score'] - min_score) / (max_score-min_score)) * (new_max - new_min) + new_min

# Display the updated DataFrame with the mapped pico_score
fico_score_for_each_grid = merged_data[['GRID_CODE', 'pico_score', 'fico_score',
                                        'precision_factor', 'inclusion_factor',
                                        'consistency_factor']]

# Display the head of the new DataFrame
fico_score_for_each_grid.head()

In [ ]:
merged_data['pico_score'].describe()

In [ ]:
merged_data['fico_score'].describe()

In [ ]:
# Check the minimum and maximum pico_score
min_score = merged_data['pico_score'].min()
max_score = merged_data['pico_score'].max()

# Print out the minimum and maximum to ensure they are different
print(f"Min PICO Score: {min_score}")
print(f"Max PICO Score: {max_score}")

# If the min and max are too close, adjust the range
if min_score == max_score:
    print("All PICO scores are the same. Mapping will result in the same value for all.")
else:
    # Define the new range (300 to 800)
    new_min = 300
    new_max = 800

    # Apply the mapping formula to map pico_score to the range of 300 to 800
    merged_data['pico_score_mapped'] = ((merged_data['pico_score'] - min_score) / (max_score - min_score)) * (new_max - new_min) + new_min

    # Display the updated DataFrame with the mapped pico_score
    pico_score_for_each_grid = merged_data[['GRID_CODE', 'pico_score', 'pico_score_mapped',
                                            'precision_factor', 'inclusion_factor',
                                            'consistency_factor']]

    # Display the head of the new DataFrame
    print(pico_score_for_each_grid.head())

In [ ]:
pico_score_for_each_grid['pico_score'].min()

In [ ]:
pico_score_for_each_grid['pico_score'].max()

In [ ]:
# Define the classification based on pico_score_mapped ranges
conditions = [
    (merged_data['fico_score'] < 580),
    (merged_data['fico_score'] >= 580) & (merged_data['fico_score'] <= 669),
    (merged_data['fico_score'] >= 670) & (merged_data['fico_score'] <= 739),
    (merged_data['fico_score'] >= 740) & (merged_data['fico_score'] <= 799),
    (merged_data['fico_score'] >= 800)
]

# Corresponding labels for each range
labels = ['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional']

# Apply the classification to create a new column pico_score_class
merged_data['fico_class'] = pd.cut(merged_data['fico_score'],
                                         bins=[-float("inf"), 580, 669, 739, 799, float("inf")],
                                         labels=labels)

# Display the updated DataFrame to verify the classification
fico_score_class_for_each_grid = merged_data[['GRID_CODE', 'fico_score', 'fico_class']]
fico_score_class_for_each_grid.head()

In [ ]:
fico_score_class_for_each_grid[fico_score_class_for_each_grid['fico_class'] == 'Poor'].head()

In [ ]:
fico_score_class_for_each_grid[fico_score_class_for_each_grid['fico_class'] == 'Good'].shape

In [ ]:
fico_score_class_for_each_grid[fico_score_class_for_each_grid['fico_class'] == 'Fair'].shape

In [ ]:
fico_score_class_for_each_grid[fico_score_class_for_each_grid['fico_class'] == 'Very Good'].shape

In [ ]:
fico_score_class_for_each_grid[fico_score_class_for_each_grid['fico_class'] == 'Exceptional'].shape

In [ ]:
location_data= pd.read_excel("/content/drive/MyDrive/Grid2023LateSeasonOromia.xlsx")
location_data = location_data[0:]
location_data.columns = location_data.iloc[0]
location_data = location_data[1:]
location_data.head()
selected_indices = [i for i in range(0, 7) if i not in [1,2, 6,]]
selected_columns = location_data.columns[selected_indices]
location_data = location_data[selected_columns]

location_data.head()

In [ ]:
!pip install shapely
from shapely.geometry import Polygon

coordinates=location_data[['X_LL','Y_LL']]
# coordinates=location_data[['X_LL','Y_LL']]
pol=[]
xx= coordinates['X_LL']
yy= coordinates['Y_LL']
for i, x in enumerate(xx):
    pol.append([xx.iloc[i],yy.iloc[i]])
polygon_all = Polygon(pol) # Now Polygon is defined and can be used
xp,yp = polygon_all.exterior.xy
plt.plot(xp,yp)
# Set plot title and labels
plt.title('Grids covered by our data collection process by location')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
fig =plt.plot(xp, yp, color='green')
fig = plt.gcf()

# Plot the data
plt.plot(xp, yp, color='green')

# Save the figure
fig.savefig("map.png")

# Display the plot
plt.show()

In [ ]:
# Convert coordinates to GeoDataFrame
location_data['geometry'] = location_data.apply(lambda row: Point(row['X_LL'], row['Y_LL']), axis=1)
gdf = gpd.GeoDataFrame(location_data, geometry='geometry', crs="EPSG:4326")

gdf = gdf.merge(merged_data[['GRID_CODE', 'fico_score']], on='GRID_CODE', how='left') # Adjust 'on' if needed

# Add a new column 'fico_score_class' to gdf and populate it using the 'fico_score' column
gdf['fico_score_class'] = gdf['fico_score'].apply(assign_fico_class) # Apply the classification to the 'fico_score' column in gdf


# Define colors for each FICO class
fico_colors = {
    'Poor': 'red',
    'Fair': 'orange',
    'Good': 'green',
    'Very Good': 'aqua',
    'Exceptional': 'blue'
}

# Plot FICO classes based on color coding
fig, ax = plt.subplots(figsize=(12, 10))

for fico_class, color in fico_colors.items():
    # Filter the GeoDataFrame for the current FICO class
    subset_gdf = gdf[gdf['fico_score_class'] == fico_class]

    # Check if the subset is not empty before plotting
    if not subset_gdf.empty:
        subset_gdf.plot(
            ax=ax,
            color=color,
            markersize=5,
            label=fico_class,
            alpha=0.7  # Transparency for overlapping points
        )

# Customize plot
plt.legend(title="FICO Class", loc="upper left")
plt.title("FICO Class Distribution Across Oromia Region")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True, linestyle="--", alpha=0.5)
# Save the figure
fig.savefig("fico_class_map.png")
plt.tight_layout()
plt.show()

In [ ]:
!pip install lime
import lime
import lime.lime_tabular
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [ ]:
# Assume merged_data has been processed and contains necessary features
# Define features and target
features = ['precision_factor', 'inclusion_factor', 'consistency_factor']  # Add other relevant features
X = merged_data[features]
y = merged_data['fico_class']  # Target variable with classifications: Poor, Fair, Good, etc.

# Split the data (if needed) and train a classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Initialize LIME
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=features,
    class_names=['Poor', 'Good', 'Good', 'Very Good', 'Exceptional'],
    mode='classification'
)

# Select a sample to explain
i = 0  # Index of the sample in X_test
exp = explainer.explain_instance(
    data_row=X_test.values[i],
    predict_fn=model.predict_proba  # Probability prediction function of the model
)

# Display the explanation
exp.show_in_notebook(show_table=True)
# Display and save the explanation plot
fig = exp.as_pyplot_figure()
fig.savefig("lime.png", format="png")

In [ ]:
# Ensure fico_class column is correctly labeled in merged_data
# Define features and target
features = ['precision_factor', 'inclusion_factor', 'consistency_factor']  # Add other relevant features
X = merged_data[features]
y = merged_data['fico_class']  # Target variable with classifications: Poor, Fair, Good, etc.

# Check for classes with only one sample
class_counts = y.value_counts()
classes_with_one_sample = class_counts[class_counts == 1].index.tolist()

# If there are classes with only one sample, handle them (e.g., remove or oversample)
if classes_with_one_sample:
    print(f"Classes with only one sample: {classes_with_one_sample}")
    X = merged_data[features]
    y = merged_data['fico_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Initialize and train the classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Initialize LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=features,
    class_names=['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional'],  # The FICO classes
    mode='classification'
)

# Select a sample from the test set to explain (change index if needed)
i = 10  # Index of the sample in X_test
exp = explainer.explain_instance(
    data_row=X_test.values[i],
    predict_fn=model.predict_proba  # Probability prediction function of the model
)

# Display the explanation in the notebook
exp.show_in_notebook(show_table=True)

# Save and display the explanation plot
fig = exp.as_pyplot_figure()
fig.savefig("lime_explanation_multi_class.png", format="png")
plt.show()

In [ ]:
merged_data = merged_data.join(user_df, on='GRID_CODE', lsuffix='_caller', rsuffix='_other')

In [ ]:
print(merged_data.columns)
print(user_df.columns)

In [ ]:
max_users = 50
user_counts = np.random.randint(1, max_users + 1, size=len(pico_score_for_each_grid))

user_data = []
index_list = []
for i, grid_code in enumerate(pico_score_for_each_grid['GRID_CODE']):
    for j in range(user_counts[i]):
        user_data.append({'GRID_CODE': grid_code, 'User': f'user_{len(user_data) + 1}'})
        index_list.append(i)
user_df = pd.DataFrame(user_data, index=index_list)
merged_data = merged_data.merge(user_df, left_on='GRID_CODE_caller', right_on='GRID_CODE', how='left')
pico_score_for_each_user = merged_data[['GRID_CODE', 'pico_score',
                                        'precision_factor', 'inclusion_factor',
                                        'consistency_factor','user_grid']]
pico_score_for_each_user.head()

In [ ]:
merged_data = merged_data.join(user_df, lsuffix='_caller', rsuffix='_other')
merged_data.head()

In [ ]:
pico_score_for_each_grid.tail(30)

In [ ]:
adjustment_factor = 0.97
merged_data['final_pico_score'] = merged_data['pico_score']*adjustment_factor
pico_score_for_each_grid = merged_data[['GRID_CODE', 'final_pico_score',
                                        'precision_factor', 'inclusion_factor',
                                        'consistency_factor']]
pico_score_for_each_grid.head()

In [ ]:
pico_score_for_each_grid.tail(30)

In [ ]:
import pickle
NDVI_Predictor = pickle.load(open('/content/NDVI_Predictor.pkl', 'rb'))

In [ ]:
NDVI_Predictor.predict()